## **문제 1-3 : FewShotPromptTemplate과 시스템 메시지 활용**

In [13]:
#문제 1-3
import os
import textwrap
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import (
    FewShotChatMessagePromptTemplate,
    ChatPromptTemplate
)

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

llm = ChatOpenAI(
    #api_key=os.getenv("GROQ_API_KEY"), # 실제 API 키 변수명에 맞게 변경하세요.
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

In [7]:
# 1. 예시 데이터 (최소 3개 포함, 다양한 분야)
examples = [
    {
        "news": "삼성전자가 내년 초에 자체적으로 개발한 인공지능(AI) 가속기를 처음으로 출시할 예정이다. 이는 AI 반도체 시장에서 지배적인 위치를 차지하고 있는 엔비디아의 독점을 도전하고, 세계 최고의 반도체 제조업체로서의 지위를 다시 확립하려는 삼성전자의 노력으로 해석된다.",
        "keywords": "삼성전자, 인공지능, 엔비디아"
    },
    {
        "news": "세계보건기구(WHO)는 최근 새로운 건강 위기에 대응하기 위해 국제 협력의 중요성을 강조했다. 전염병 대응 역량의 강화와 글로벌 보건 시스템의 개선이 필요하다고 발표했다.",
        "keywords": "세계보건기구, 건강위기, 국제협력"
    },
    {
        "news": "정부가 다음 달부터 새로운 부동산 정책을 시행한다고 발표했다. 주택 공급 확대와 투기 억제를 목표로 하며, 서민 주거 안정에 기여할 것으로 기대된다.",
        "keywords": "정부, 부동산정책, 주택공급"
    },
    {
        "news": "최근 K-POP 아이돌 그룹 '블랙핑크'가 빌보드 차트에서 신기록을 달성하며 글로벌 팬들의 뜨거운 반응을 얻고 있다. 이들은 독특한 음악 스타일과 퍼포먼스로 전 세계 음악 시장에 큰 영향력을 미치고 있다.",
        "keywords": "블랙핑크, K-POP, 빌보드"
    }
]

In [8]:
# 2. 예시 프롬프트 (대화형)
example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{news}"),
    ("ai", "키워드: {keywords}") # LLM이 출력할 형식에 맞춰 '키워드: ' 접두사 포함
])

In [9]:
# 3. Few-Shot 프롬프트
# 'input_variables' 줄을 제거합니다. FewShotChatMessagePromptTemplate이 자동으로 추론합니다.
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples
)

In [10]:
# 4. 최종 프롬프트
final_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 뉴스 기사에서 핵심 키워드 3개를 추출하는 전문가입니다. 주어진 뉴스 텍스트에서 가장 중요하고 핵심적인 키워드 3개를 쉼표(,)로 구분하여 '키워드: ' 형식으로 출력하세요."),
    few_shot_prompt, # Few-Shot 예시들이 이 위치에 삽입됩니다.
    ("human", "{news_text}") # 사용자가 제공할 뉴스 텍스트를 위한 변수
])


In [16]:
# 5. 체인 구성
keyword_extraction_chain = final_prompt | llm | StrOutputParser()

# 6. 테스트 뉴스
test_news = """제미나이 2.0 플래시는 현재 구글 AI 스튜디오(Google AI Studio) 및 버텍스 AI(Vertex AI)에서 제미나이 API를 통해 개발자에게 실험 모델로 제공됩니다. 모든 개발자는 멀티모달 입력 및 텍스트 출력을 사용할 수 있으며, 텍스트 음성 변환(text-to-speech) 및 네이티브 이미지 생성은 일부 파트너들을 대상으로 제공됩니다. 내년 1월에는 더 많은 모델 사이즈와 함께 일반에 공개될 예정입니다."""

# 7. 실행 및 결과 출력
print("--- 뉴스 키워드 추출 테스트 ---")
# textwrap.fill()을 사용하여 뉴스를 70자 너비로 줄 바꿈하여 출력
print(f"테스트 뉴스:\n{textwrap.fill(test_news, width=70)}\n")
extracted_keywords = keyword_extraction_chain.invoke({"news_text": test_news})

print(f"키워드: {extracted_keywords}")

--- 뉴스 키워드 추출 테스트 ---
테스트 뉴스:
제미나이 2.0 플래시는 현재 구글 AI 스튜디오(Google AI Studio) 및 버텍스 AI(Vertex AI)에서
제미나이 API를 통해 개발자에게 실험 모델로 제공됩니다. 모든 개발자는 멀티모달 입력 및 텍스트 출력을 사용할 수 있으며,
텍스트 음성 변환(text-to-speech) 및 네이티브 이미지 생성은 일부 파트너들을 대상으로 제공됩니다. 내년 1월에는
더 많은 모델 사이즈와 함께 일반에 공개될 예정입니다.

키워드: 키워드: 제미나이, 구글, AI
